In [1]:
import tensorflow

from sklearn.preprocessing import normalize
import pandas as pd
import numpy as np
import tensorflow as tf
import random
df=pd.read_csv('C:/Users/jira_/Desktop/new_stresult22-15-22.csv',header=None)
d = df.values
ntotal=len(df)

# we do not renormalize it 
#d = normalize(d, axis=0, norm='l2')


resultu = []
np.random.rand(4)
# Return 100 results (for instance)
for i in range(ntotal):
    
    res = random.random()
    if res < 0.1:
        resultu.append(1)
    elif res < 0.2 and res>=0.1:
        resultu.append(2)
    elif res < 0.3 and res>=0.2:
        resultu.append(3)
    elif res < 0.4 and res>=0.3:
        resultu.append(4)
    elif res < 0.5 and res>=0.4:
        resultu.append(5)
    elif res < 0.6 and res>=0.5:
        resultu.append(6)
    elif res < 0.7 and res>=0.6:
        resultu.append(8)
    elif res < 0.8 and res>=0.7:
        resultu.append(8)
    else:
        resultu.append(9)
resultu=np.array(resultu)
trainset=[]
for i in range(1,7):
    toinsert=d[resultu==i].astype(np.float32)
    trainset.append(toinsert)
validationset=d[resultu==8].astype(np.float32)
testset=d[resultu==9].astype(np.float32)


#x = data

# Following Hinton-Salakhutdinov Architecture

# 3 hidden layers for encoder
n_encoder_h_1 = 13
n_encoder_h_2 = 9



#n_encoder_h_5 = 10


# 3 hidden layers for decoder
#n_decoder_h_1 = 10
n_decoder_h_1 = 14
n_decoder_h_2 = 19




# Parameters
learning_rate = 0.005

#batch_size = 7
display_step = 1

total_batch=6
training_epochs = 5000

In [2]:
#we store the variables here
log_files_path = 'C:/Users/jira_/Desktop/new_st15-10-15/tmp/model.ckpt'
def layer_batch_normalization(x, n_out, phase_train):
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - n_out: integer, depth of input maps - number of sample in the batch 
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - batch-normalized maps   
    
    
    """

    beta_init = tf.constant_initializer(value=0.0, dtype=tf.float32)
    beta = tf.get_variable("beta", [n_out], initializer=beta_init)
    
    gamma_init = tf.constant_initializer(value=1.0, dtype=tf.float32)
    gamma = tf.get_variable("gamma", [n_out], initializer=gamma_init)

    #tf.nn.moment: https://www.tensorflow.org/api_docs/python/tf/nn/moments
    #calculate mean and variance of x
    batch_mean, batch_var = tf.nn.moments(x, [0], name='moments')
    
    #tf.train.ExponentialMovingAverage:
    #https://www.tensorflow.org/api_docs/python/tf/train/ExponentialMovingAverage
    #Maintains moving averages of variables by employing an exponential decay.
    ema = tf.train.ExponentialMovingAverage(decay=0.9)
    ema_apply_op = ema.apply([batch_mean, batch_var])
    ema_mean, ema_var = ema.average(batch_mean), ema.average(batch_var)
    
    def mean_var_with_update():
        with tf.control_dependencies([ema_apply_op]):
            return tf.identity(batch_mean), tf.identity(batch_var)
        
    #tf.cond: https://www.tensorflow.org/api_docs/python/tf/cond
    #Return true_fn() if the predicate pred is true else false_fn()
    mean, var = tf.cond(phase_train, mean_var_with_update, lambda: (ema_mean, ema_var))
    
    
    #Here, we changesd the shape of x into [[[x1]],[[x2] ],....]

    reshaped_x = tf.reshape(x, [-1, 1, 1, n_out])
    normed = tf.nn.batch_norm_with_global_normalization(reshaped_x, mean, var, beta, gamma, 1e-3, True)
    
    return tf.reshape(normed, [-1, n_out])

def layer(x, weight_shape, bias_shape, phase_train):
    
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - weight_shape: shape the the weight maxtrix
        - bias_shape: shape of the bias vector
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector of the layer after the matrix multiplication and non linear transformation
    """
    
    #initialize weights
    weight_init = tf.random_normal_initializer(stddev=(1.0/weight_shape[0])**0.5)
    W = tf.get_variable("W", weight_shape, initializer=weight_init)
    
    bias_init = tf.constant_initializer(value=0)
    b = tf.get_variable("b", bias_shape, initializer=bias_init)

    logits = tf.matmul(x, W) + b
    
    #apply the non-linear function after the batch normalization
    return tf.nn.sigmoid(layer_batch_normalization(logits, weight_shape[1], phase_train))
def encoder(x, n_code, phase_train):
    
    """
    Defines the network encoder part
    input:
        - x: input vector of the encoder
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder)
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector: reduced dimension
    """
    
    with tf.variable_scope("encoder"):
        
        
        with tf.variable_scope("code"):
            output = layer(x, [15, n_code], [n_code], phase_train)


    return output

def decoder(x, n_code, phase_train):
    """
    Defines the network encoder part
    input:
        - x: input vector of the decoder - reduced dimension vector
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder) 
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector: reconstructed dimension of the initial vector
    """
    
    with tf.variable_scope("decoder"):
        
   
        with tf.variable_scope("output"):
            output = layer(x, [ n_code, 15], [15], phase_train)

    return output
def loss(output, x):
    """
    Compute the loss of the auto-encoder
    
    intput:
        - output: the output of the decoder
        - x: true value of the sample batch - this is the input of the encoder
        
        the two have the same shape (batch_size * num_of_classes)
    output:
        - loss: loss of the corresponding batch (scalar tensor)
    
    """
    
    with tf.variable_scope("training"):
        
        l2 = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x)), 1))
        train_loss = tf.reduce_mean(l2)
        train_summary_op = tf.summary.scalar("train_cost", train_loss)
        return train_loss, train_summary_op
def training(cost, global_step):
    """
    defines the necessary elements to train the network
    
    intput:
        - cost: the cost is the loss of the corresponding batch
        - global_step: number of batch seen so far, it is incremented by one 
        each time the .minimize() function is called
    """
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-08, use_locking=False, name='Adam')
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.005)
    train_op = optimizer.minimize(cost, global_step=global_step)
    return train_op
def evaluate(output, x):
    """
    evaluates the accuracy on the validation set 
    input:
        -output: prediction vector of the network for the validation set
        -x: true value for the validation set
    output:
        - val_loss: loss of the autoencoder
        - in_image_op: input image 
        - out_image_op:reconstructed image 
        - val_summary_op: summary of the loss
    """
    
    with tf.variable_scope("validation"):
        
        #in_image_op = image_summary("input_image", x)
        
        #out_image_op = image_summary("output_image", output)
        
        l2_norm = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x, name="val_diff")), 1))
        
        val_loss = tf.reduce_mean(l2_norm)
        
        val_summary_op = tf.summary.scalar("val_cost", val_loss)
        
        #return val_loss, in_image_op, out_image_op, val_summary_op
        return val_loss,  val_summary_op
"""
def image_summary(label, tensor):
    #tf.summary.image: https://www.tensorflow.org/api_docs/python/tf/summary/image
    #Outputs a Summary protocol buffer with images.

    tensor_reshaped = tf.reshape(tensor, [-1, 19, 1, 1])
    return tf.summary.image(label, tensor_reshaped)
"""
if __name__ == '__main__':
    print('we begin')

    #if a python file, please use the 4 lines bellow and comment the "n_code = '2'"
    #parser = argparse.ArgumentParser(description='Autoencoder')
    #parser.add_argument('n_code', nargs=1, type=str)
    #args = parser.parse_args(['--help'])
    #n_code = args.n_code[0]
    
    #if a jupyter file, please comment the 4 above and use the one bellow
    n_code = '10'
    
    #feel free to change with your own 
    #log_files_path = r'C:\Users\yy2895\Desktop\pys'
    
    with tf.Graph().as_default():

        with tf.variable_scope("autoencoder_model"):

            #the input variables are first define as placeholder 
            # a placeholder is a variable/data which will be assigned later 
            # image vector & label, phase_train is a boolean 
            x = tf.placeholder("float", [None, 15]) # MNIST data image of shape 28*28=784
            
            phase_train = tf.placeholder(tf.bool)
            
            #define the encoder 
            code = encoder(x, int(n_code), phase_train)
            
            #define the decoder
            output = decoder(code, int(n_code), phase_train)
            
            #compute the loss 
            cost, train_summary_op = loss(output, x)

            #initialize the value of the global_step variable 
            # recall: it is incremented by one each time the .minimise() is called
            global_step = tf.Variable(0, name='global_step', trainable=False)

            train_op = training(cost, global_step)

            #evaluate the accuracy of the network (done on a validation set)
            #eval_op, in_image_op, out_image_op, val_summary_op = evaluate(output, x)
            eval_op, val_summary_op = evaluate(output, x)

            summary_op = tf.summary.merge_all()

            #save and restore variables to and from checkpoints.
            #saver = tf.train.Saver(max_to_keep=200)
            saver = tf.train.Saver()

            #defines a session
            sess = tf.Session()

            # summary writer
            #https://www.tensorflow.org/api_docs/python/tf/summary/FileWriter
            #train_writer = tf.summary.FileWriter(log_files_path+'mnist_autoencoder_hidden_' + n_code + '_logs/', graph=sess.graph)

            #val_writer   = tf.summary.FileWriter(log_files_path+'mnist_autoencoder_hidden_' + n_code + '_logs/', graph=sess.graph)

            #initialization of the variables
            init_op = tf.global_variables_initializer()

            sess.run(init_op)
            currentmin=10000000000
            currentmin_index=-1
            error_path=[]
            togive2=[]
            # Training cycle
            for epoch in range(training_epochs):

                avg_cost = 0.
                #total_batch = int(numberofsamples/batch_size)
                
                # Loop over all batches
                for i in range(total_batch):
                    
                    minibatch_x= trainset[i]
                    
                    # Fit training using batch data
                    #the training is done using the training dataset
                    _, new_cost, train_summary = sess.run([train_op, cost, train_summary_op], feed_dict={x: minibatch_x, phase_train: True})
                    
                    #train_writer.add_summary(train_summary, sess.run(global_step))
                    
                    # Compute average loss
                    avg_cost += new_cost/total_batch
                    
                
                # Display logs per epoch step
                if epoch % display_step == 0:
                    print("Epoch:", '%04d' % (epoch+1), "cost =", "{:0.9f}".format(avg_cost))

                    #the accuracy is evaluated using the validation dataset
                    #train_writer.add_summary(train_summary, sess.run(global_step))

                    #validation_loss, in_image, out_image, val_summary = sess.run([eval_op, in_image_op, out_image_op, val_summary_op], feed_dict={x: validationset, phase_train: False})
                    validation_loss,  val_summary = sess.run([eval_op, val_summary_op], feed_dict={x: validationset, phase_train: False})
                    #val_writer.add_summary(in_image, sess.run(global_step))
                    #val_writer.add_summary(out_image, sess.run(global_step))
                    #val_writer.add_summary(val_summary, sess.run(global_step))
                    print("Validation Loss:", validation_loss)
                    error_path.append(validation_loss)
                    
                    if validation_loss<currentmin:
                        currentmin=validation_loss
                        currentmin_index=epoch
                        saver.save(sess, 'C:/Users/jira_/Desktop/new_st15-10-15/tmp/model.ckpt')
                        togive8=[]
                        for i in range(len(d)):
                            any_image = d[i].reshape(-1,15)
                            output_any_image = sess.run(code,feed_dict={x:any_image,phase_train: False})
                            togive8.append(output_any_image)
                            
                        togive2=togive8
                    #save to use later
                    #https://www.tensorflow.org/api_docs/python/tf/train/Saver
                    #saver.save(sess, log_files_path+'model-checkpoint', global_step=global_step)
                    #saver.save(sess, 'C:/Users/yy2895/Desktop/st19-14-19/tmp/model.ckpt')
                    

            print("Optimization Finished!")

            test_loss = sess.run(eval_op, feed_dict={x: testset, phase_train: False})
            #nps_loss = sess.run(eval_op, feed_dict={x: mnist.validation.images, phase_train: False})
            print("Test Loss:", test_loss)

we begin
Epoch: 0001 cost = 1.513321757
Validation Loss: 1.9038212
Epoch: 0002 cost = 1.499282976
Validation Loss: 1.6948574
Epoch: 0003 cost = 1.484800597
Validation Loss: 1.5409014
Epoch: 0004 cost = 1.470915655
Validation Loss: 1.4839457
Epoch: 0005 cost = 1.455825686
Validation Loss: 1.4733157
Epoch: 0006 cost = 1.437972585
Validation Loss: 1.4647849
Epoch: 0007 cost = 1.419131100
Validation Loss: 1.4470496
Epoch: 0008 cost = 1.400912682
Validation Loss: 1.4108238
Epoch: 0009 cost = 1.386114617
Validation Loss: 1.3681378
Epoch: 0010 cost = 1.373543501
Validation Loss: 1.3429765
Epoch: 0011 cost = 1.359403531
Validation Loss: 1.3288387
Epoch: 0012 cost = 1.344425778
Validation Loss: 1.3014458
Epoch: 0013 cost = 1.329856157
Validation Loss: 1.2643517
Epoch: 0014 cost = 1.315769792
Validation Loss: 1.2478309
Epoch: 0015 cost = 1.303049803
Validation Loss: 1.2545693
Epoch: 0016 cost = 1.291756511
Validation Loss: 1.3311175
Epoch: 0017 cost = 1.281332513
Validation Loss: 1.3653537
Epoch

Epoch: 0142 cost = 0.561410874
Validation Loss: 0.6189037
Epoch: 0143 cost = 0.557593097
Validation Loss: 0.62072355
Epoch: 0144 cost = 0.553795040
Validation Loss: 0.60090107
Epoch: 0145 cost = 0.550015539
Validation Loss: 0.6137983
Epoch: 0146 cost = 0.546255310
Validation Loss: 0.5986909
Epoch: 0147 cost = 0.542513927
Validation Loss: 0.61241007
Epoch: 0148 cost = 0.538793365
Validation Loss: 0.5723006
Epoch: 0149 cost = 0.535091549
Validation Loss: 0.5635214
Epoch: 0150 cost = 0.531410813
Validation Loss: 0.5396228
Epoch: 0151 cost = 0.527749916
Validation Loss: 0.534012
Epoch: 0152 cost = 0.524109681
Validation Loss: 0.5342271
Epoch: 0153 cost = 0.520490448
Validation Loss: 0.48955172
Epoch: 0154 cost = 0.516891559
Validation Loss: 0.52635616
Epoch: 0155 cost = 0.513313750
Validation Loss: 0.5127002
Epoch: 0156 cost = 0.509757936
Validation Loss: 0.5244311
Epoch: 0157 cost = 0.506223549
Validation Loss: 0.51559937
Epoch: 0158 cost = 0.502711495
Validation Loss: 0.4988797
Epoch: 01

Epoch: 0284 cost = 0.223054772
Validation Loss: 0.2598842
Epoch: 0285 cost = 0.221736694
Validation Loss: 0.23297349
Epoch: 0286 cost = 0.220428454
Validation Loss: 0.22826599
Epoch: 0287 cost = 0.219129379
Validation Loss: 0.22540407
Epoch: 0288 cost = 0.217839907
Validation Loss: 0.2629602
Epoch: 0289 cost = 0.216559862
Validation Loss: 0.23098883
Epoch: 0290 cost = 0.215289138
Validation Loss: 0.21773538
Epoch: 0291 cost = 0.214027350
Validation Loss: 0.2474598
Epoch: 0292 cost = 0.212775121
Validation Loss: 0.21271282
Epoch: 0293 cost = 0.211531195
Validation Loss: 0.23987173
Epoch: 0294 cost = 0.210296802
Validation Loss: 0.20969741
Epoch: 0295 cost = 0.209071276
Validation Loss: 0.24218269
Epoch: 0296 cost = 0.207854311
Validation Loss: 0.21432619
Epoch: 0297 cost = 0.206646018
Validation Loss: 0.20776576
Epoch: 0298 cost = 0.205446606
Validation Loss: 0.24446937
Epoch: 0299 cost = 0.204255561
Validation Loss: 0.21250778
Epoch: 0300 cost = 0.203073144
Validation Loss: 0.20888813


Epoch: 0426 cost = 0.102676605
Validation Loss: 0.15528452
Epoch: 0427 cost = 0.102160959
Validation Loss: 0.11482209
Epoch: 0428 cost = 0.101648380
Validation Loss: 0.104742415
Epoch: 0429 cost = 0.101139123
Validation Loss: 0.16094854
Epoch: 0430 cost = 0.100633219
Validation Loss: 0.116053104
Epoch: 0431 cost = 0.100130604
Validation Loss: 0.10165631
Epoch: 0432 cost = 0.099631095
Validation Loss: 0.15518148
Epoch: 0433 cost = 0.099134814
Validation Loss: 0.115794905
Epoch: 0434 cost = 0.098641761
Validation Loss: 0.10869916
Epoch: 0435 cost = 0.098151808
Validation Loss: 0.105290376
Epoch: 0436 cost = 0.097665093
Validation Loss: 0.101007074
Epoch: 0437 cost = 0.097181348
Validation Loss: 0.15188438
Epoch: 0438 cost = 0.096700920
Validation Loss: 0.11244981
Epoch: 0439 cost = 0.096223616
Validation Loss: 0.104513556
Epoch: 0440 cost = 0.095749259
Validation Loss: 0.10337295
Epoch: 0441 cost = 0.095278028
Validation Loss: 0.10529794
Epoch: 0442 cost = 0.094809676
Validation Loss: 0.

Epoch: 0571 cost = 0.054787208
Validation Loss: 0.11410107
Epoch: 0572 cost = 0.054601714
Validation Loss: 0.075560614
Epoch: 0573 cost = 0.054417478
Validation Loss: 0.073451385
Epoch: 0574 cost = 0.054234921
Validation Loss: 0.06985861
Epoch: 0575 cost = 0.054053692
Validation Loss: 0.06800792
Epoch: 0576 cost = 0.053874040
Validation Loss: 0.0667474
Epoch: 0577 cost = 0.053695885
Validation Loss: 0.06627438
Epoch: 0578 cost = 0.053519010
Validation Loss: 0.08093958
Epoch: 0579 cost = 0.053343582
Validation Loss: 0.08128146
Epoch: 0580 cost = 0.053169596
Validation Loss: 0.08329164
Epoch: 0581 cost = 0.052997097
Validation Loss: 0.08928278
Epoch: 0582 cost = 0.052825989
Validation Loss: 0.083384685
Epoch: 0583 cost = 0.052656239
Validation Loss: 0.07964891
Epoch: 0584 cost = 0.052487805
Validation Loss: 0.0723058
Epoch: 0585 cost = 0.052320666
Validation Loss: 0.057851918
Epoch: 0586 cost = 0.052155141
Validation Loss: 0.056514066
Epoch: 0587 cost = 0.051990831
Validation Loss: 0.054

...

Validation Loss: 0.05608443
Epoch: 4292 cost = 0.023936631
Validation Loss: 0.06983111
Epoch: 4293 cost = 0.023927932
Validation Loss: 0.057805926
Epoch: 4294 cost = 0.023919112
Validation Loss: 0.044839777
Epoch: 4295 cost = 0.023910778
Validation Loss: 0.045878682
Epoch: 4296 cost = 0.023902782
Validation Loss: 0.042088684
Epoch: 4297 cost = 0.023896387
Validation Loss: 0.09075102
Epoch: 4298 cost = 0.023891848
Validation Loss: 0.104346335
Epoch: 4299 cost = 0.023889795
Validation Loss: 0.10946309
Epoch: 4300 cost = 0.023892137
Validation Loss: 0.10963066
Epoch: 4301 cost = 0.023899291
Validation Loss: 0.10125588
Epoch: 4302 cost = 0.023913342
Validation Loss: 0.047868915
Epoch: 4303 cost = 0.023931519
Validation Loss: 0.0597373
Epoch: 4304 cost = 0.023953406
Validation Loss: 0.039169613
Epoch: 4305 cost = 0.023976491
Validation Loss: 0.04217667
Epoch: 4306 cost = 0.023999204
Validation Loss: 0.05899282
Epoch: 4307 cost = 0.024019617
Validation Loss: 0.05834324
Epoch: 4308 cost = 0.0

Validation Loss: 0.042107344
Epoch: 4458 cost = 0.023829632
Validation Loss: 0.04082851
Epoch: 4459 cost = 0.023825738
Validation Loss: 0.052112505
Epoch: 4460 cost = 0.023827180
Validation Loss: 0.07620866
Epoch: 4461 cost = 0.023837259
Validation Loss: 0.09213033
Epoch: 4462 cost = 0.023856847
Validation Loss: 0.087088294
Epoch: 4463 cost = 0.023888329
Validation Loss: 0.05650278
Epoch: 4464 cost = 0.023923966
Validation Loss: 0.052521802
Epoch: 4465 cost = 0.023954007
Validation Loss: 0.07454047
Epoch: 4466 cost = 0.023965482
Validation Loss: 0.046474032
Epoch: 4467 cost = 0.023953748
Validation Loss: 0.051938634
Epoch: 4468 cost = 0.023926152
Validation Loss: 0.081096314
Epoch: 4469 cost = 0.023896041
Validation Loss: 0.06948278
Epoch: 4470 cost = 0.023870471
Validation Loss: 0.06447901
Epoch: 4471 cost = 0.023851727
Validation Loss: 0.04632414
Epoch: 4472 cost = 0.023837434
Validation Loss: 0.05593003
Epoch: 4473 cost = 0.023824332
Validation Loss: 0.07802791
Epoch: 4474 cost = 0.

Validation Loss: 0.038780563
Epoch: 4622 cost = 0.023820377
Validation Loss: 0.039850686
Epoch: 4623 cost = 0.023822644
Validation Loss: 0.032664705
Epoch: 4624 cost = 0.023820202
Validation Loss: 0.034089413
Epoch: 4625 cost = 0.023813572
Validation Loss: 0.06218561
Epoch: 4626 cost = 0.023803656
Validation Loss: 0.043903403
Epoch: 4627 cost = 0.023792411
Validation Loss: 0.03843363
Epoch: 4628 cost = 0.023778025
Validation Loss: 0.054365814
Epoch: 4629 cost = 0.023764332
Validation Loss: 0.047686756
Epoch: 4630 cost = 0.023750823
Validation Loss: 0.03927941
Epoch: 4631 cost = 0.023738129
Validation Loss: 0.04371403
Epoch: 4632 cost = 0.023728756
Validation Loss: 0.044613987
Epoch: 4633 cost = 0.023720999
Validation Loss: 0.042066664
Epoch: 4634 cost = 0.023719105
Validation Loss: 0.041444074
Epoch: 4635 cost = 0.023720966
Validation Loss: 0.037489545
Epoch: 4636 cost = 0.023732048
Validation Loss: 0.030064074
Epoch: 4637 cost = 0.023752185
Validation Loss: 0.04067722
Epoch: 4638 cost

Epoch: 4785 cost = 0.023649380
Validation Loss: 0.06728857
Epoch: 4786 cost = 0.023668183
Validation Loss: 0.053891797
Epoch: 4787 cost = 0.023684621
Validation Loss: 0.070354804
Epoch: 4788 cost = 0.023697363
Validation Loss: 0.07759028
Epoch: 4789 cost = 0.023705986
Validation Loss: 0.089353316
Epoch: 4790 cost = 0.023710231
Validation Loss: 0.08747685
Epoch: 4791 cost = 0.023710450
Validation Loss: 0.08847637
Epoch: 4792 cost = 0.023706201
Validation Loss: 0.0705465
Epoch: 4793 cost = 0.023699949
Validation Loss: 0.048315357
Epoch: 4794 cost = 0.023691123
Validation Loss: 0.033580527
Epoch: 4795 cost = 0.023681027
Validation Loss: 0.04445109
Epoch: 4796 cost = 0.023670159
Validation Loss: 0.033518124
Epoch: 4797 cost = 0.023658129
Validation Loss: 0.049356308
Epoch: 4798 cost = 0.023647587
Validation Loss: 0.07363428
Epoch: 4799 cost = 0.023638737
Validation Loss: 0.074873075
Epoch: 4800 cost = 0.023631555
Validation Loss: 0.07142593
Epoch: 4801 cost = 0.023628062
Validation Loss: 0

Epoch: 4949 cost = 0.023591508
Validation Loss: 0.062434636
Epoch: 4950 cost = 0.023600180
Validation Loss: 0.05274488
Epoch: 4951 cost = 0.023605279
Validation Loss: 0.09547746
Epoch: 4952 cost = 0.023607167
Validation Loss: 0.09594434
Epoch: 4953 cost = 0.023606129
Validation Loss: 0.08993869
Epoch: 4954 cost = 0.023602848
Validation Loss: 0.06813229
Epoch: 4955 cost = 0.023597275
Validation Loss: 0.054767303
Epoch: 4956 cost = 0.023590251
Validation Loss: 0.122117594
Epoch: 4957 cost = 0.023582652
Validation Loss: 0.13216609
Epoch: 4958 cost = 0.023573724
Validation Loss: 0.092449464
Epoch: 4959 cost = 0.023566095
Validation Loss: 0.057678998
Epoch: 4960 cost = 0.023556905
Validation Loss: 0.04671155
Epoch: 4961 cost = 0.023551313
Validation Loss: 0.051598717
Epoch: 4962 cost = 0.023548098
Validation Loss: 0.05522164
Epoch: 4963 cost = 0.023547897
Validation Loss: 0.123082615
Epoch: 4964 cost = 0.023551545
Validation Loss: 0.18414615
Epoch: 4965 cost = 0.023558274
Validation Loss: 0

In [3]:
import csv
with open('C:/Users/jira_/Desktop/new_stresult15-10-15.csv', 'w',newline='') as myfile:
     wr = csv.writer(myfile)
     for i in range(len(togive2)):
        wr.writerow(list(togive2[i][0]))